In [1]:
#!/usr/bin/env python
import subprocess
from datetime import datetime

In [2]:
'''
    Testing keepalive for TCP connections (proto=6) - connections that were kept alive for more than n seconds
'''
n = 21
p = subprocess.run("rwfilter ./mysterious-dataset.rw --proto=6 --pass=stdout  --type=all | rwcut --field=sIp,dIp,stime,etime --no-columns --column-separator=' ' --no-title",
                    capture_output=True,
                    shell=True
                )

# I have had some problems using (rwfilter with SILK_DATA_ROOTDIR even with right silk.conf configurations) 
# the start-date and end-date filtering, problems which I don't know where they come from or how to solve.

# --start-date=2014/11/24:16 --end-date=2014/11/24:17
# --start-date=2014/11/24:16 --end-date=2014/12/01:13

if p.stderr:
    raise p.stderr.decode()

out = p.stdout.decode()

In [3]:
# key = x = dates
# values = y = count
dates_in_hours = dict()
dates_in_seconds = dict()
dates_in_seconds2 = dict()
t = out.splitlines()

long_requests = dict()

for l in t:
    sip,dip,sdate,edate = l.strip().split(' ')
    sdt = datetime.strptime(sdate, '%Y/%m/%dT%H:%M:%S.%f')
    edt = datetime.strptime(edate, '%Y/%m/%dT%H:%M:%S.%f')
    
    sec_dif = edt.timestamp() - sdt.timestamp()

    if sec_dif >= n:
        key_str = "sip:{} dip:{}".format(sip,dip)
        current_val = long_requests.get(key_str)
        if current_val:
            current_duration = float(current_val.split(" ")[0].split(":")[1])
            current_count = int(current_val.split(" ")[1].split(":")[1])
            current_duration += sec_dif
            current_count += 1
            long_requests[key_str]="duration:{} count:{}".format(current_duration,current_count)
        else:
            long_requests[key_str]="duration:{} count:1".format(sec_dif)


# developed in wsl -> threading would be a pain and not worth it (even though in a normal env would make sense given the number of log entries)


In [4]:
output_dict = dict()
for dict_entry_key in long_requests:
    dict_entry = long_requests[dict_entry_key]
    current_duration = float(dict_entry.split(" ")[0].split(":")[1])
    current_count = int(dict_entry.split(" ")[1].split(":")[1])
    mean_duration = current_duration / current_count
    
    output_dict[dict_entry_key]="mean_duration:{} count:{}".format(mean_duration,current_count)

long_requests_str = str(output_dict)
long_requests_str = long_requests_str.replace("{'","{\n\t'")
long_requests_str = long_requests_str.replace("'}","'\n}")
long_requests_str = long_requests_str.replace("', '","',\n\t'")
print(str(long_requests_str))

{
	'sip:192.168.0.76 dip:192.168.0.200': 'mean_duration:27.607548359901674 count:31',
	'sip:192.168.0.200 dip:192.168.0.76': 'mean_duration:27.59506448622673 count:31',
	'sip:192.168.0.72 dip:192.168.0.200': 'mean_duration:28.161137934388787 count:29',
	'sip:192.168.0.200 dip:192.168.0.72': 'mean_duration:28.148586207422717 count:29',
	'sip:192.168.0.94 dip:192.168.0.200': 'mean_duration:49.9455217278522 count:46',
	'sip:192.168.0.200 dip:192.168.0.94': 'mean_duration:49.896586962368175 count:46',
	'sip:192.168.0.108 dip:192.168.0.200': 'mean_duration:24.61638459792504 count:13',
	'sip:192.168.0.200 dip:192.168.0.108': 'mean_duration:24.601538456403294 count:13',
	'sip:192.168.0.115 dip:192.168.0.200': 'mean_duration:76.48506796733047 count:5944',
	'sip:192.168.0.200 dip:192.168.0.115': 'mean_duration:76.48169448117389 count:5944',
	'sip:192.168.0.113 dip:192.168.0.200': 'mean_duration:92.41820149077152 count:14864',
	'sip:192.168.0.200 dip:192.168.0.113': 'mean_duration:56.04723182916

With the given output, we can say that there is not a timeout implemented in some tcp connections, or those timeouts vary from connection from connection, but some ip addresses seem to keep their connections alive as long as they can, reconnecting after they die out. These actions are suspicious, and can be mostly observed on connections that come from the IP address '192.168.0.113'.
Like so, we can say that this can be identified as a TCP Keepalive, with the atacker's ip 192.168.0.113.